<a href="https://colab.research.google.com/github/piyush26c/Advance-Data-Structure-Assignments/blob/master/Assignment_6(Vector_addition_%2B_Matrix_multiplication).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-0n3xor96
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-0n3xor96
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=df6337fbf5669d1dfe91b7f18ad31c2a21b8a4820ad7abe324d8dccf9802375c
  Stored in directory: /tmp/pip-ephem-wheel-cache-fup50q8l/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [13]:
%%cu
//Author : Piyush Rajendra Chaudhari
//Roll No: BECOC311
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#define VECTOR_SIZE 10

__global__ void vectorAddition (long long *vectorA_, long long *vectorB_, long long *vectorC_) {
	vectorC_[blockIdx.x] = vectorA_[blockIdx.x] + vectorB_[blockIdx.x];
}

void fillVector (long long *vector_) {
	for (int indx = 0; indx < VECTOR_SIZE; indx++) {
		vector_[indx] = indx;
	}
}

void printVector (long long *vector_) {
	for (int indx = 0; indx < VECTOR_SIZE; indx++) {
		printf("%lld ", vector_[indx]);
	}
}

int main(void) {
	long long *hostVectorA, *hostVectorB, *hostVectorC;
	long long *deviceVectorA, *deviceVectorB, *deviceVectorC;
	long long memorySize = VECTOR_SIZE * sizeof(long long);
	
	// Allocate space for host vectors A, B, C and insert input values
  hostVectorA = (long long *)malloc(memorySize); 
	fillVector(hostVectorA);
  hostVectorB = (long long *)malloc(memorySize); 
	fillVector(hostVectorB);
  hostVectorC = (long long *)malloc(memorySize);
	
	// Allocate space for device vectors A, B, C
  cudaMalloc((void **)&deviceVectorA, VECTOR_SIZE * sizeof(long long));
  cudaMalloc((void **)&deviceVectorB, VECTOR_SIZE *sizeof(long long));
  cudaMalloc((void **)&deviceVectorC, VECTOR_SIZE * sizeof(long long));
	
	// Copy vector data from host to device
  cudaMemcpy(deviceVectorA, hostVectorA, VECTOR_SIZE * sizeof(long long), cudaMemcpyHostToDevice);
  cudaMemcpy(deviceVectorB, hostVectorB, VECTOR_SIZE * sizeof(long long), cudaMemcpyHostToDevice);
	
	//by creating multiple blocks with single thread in it.
	vectorAddition<<<VECTOR_SIZE, 1>>>(deviceVectorA, deviceVectorB, deviceVectorC);
 
  // Copy result back to host
  cudaMemcpy(hostVectorC, deviceVectorC, VECTOR_SIZE * sizeof(long long), cudaMemcpyDeviceToHost);
 
  printf("VECTOR A : ");
	printVector(hostVectorA);
	printf("\nVECTOR B : ");
	printVector(hostVectorB);
	printf("\nVECTOR C : ");
	printVector(hostVectorC);
  free(hostVectorA); 
	free(hostVectorB); 
	free(hostVectorC);
 
  // free gpu memory
  cudaFree(deviceVectorA); 
	cudaFree(deviceVectorB); 
	cudaFree(deviceVectorC);
	
	return 0;
}

VECTOR A : 0 1 2 3 4 5 6 7 8 9 
VECTOR B : 0 1 2 3 4 5 6 7 8 9 
VECTOR C : 0 2 4 6 8 10 12 14 16 18 
